In [1]:
!pip install torch torchvision transformers pandas


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install xlrd

     ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
     ------------ --------------------------- 30.7/96.5 kB 1.3 MB/s eta 0:00:01
     ---------------- --------------------- 41.0/96.5 kB 960.0 kB/s eta 0:00:01
     -------------------------------- ----- 81.9/96.5 kB 762.6 kB/s eta 0:00:01
     ------------------------------------ - 92.2/96.5 kB 744.7 kB/s eta 0:00:01
     -------------------------------------- 96.5/96.5 kB 501.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Step 1: Install Required Libraries
# !pip install torch torchvision transformers pandas

import pandas as pd
from transformers import pipeline, BertTokenizer, BertModel
import torch
#import openpyxl

# Sample Data
df = pd.read_csv('n8n.csv')

# Step 2: Setup Sentiment Analysis and BERT Model for Embeddings
sentiment_analyzer = pipeline("sentiment-analysis")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_embedding(text):
    tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)[0]
    return embeddings.cpu().numpy()

# Step 3: Generate Personalized Emails
def generate_advanced_email(row):
    item = row['Item Purchased']
    feedback = row['Feedback']
    sentiment = sentiment_analyzer(feedback)[0]['label']
    
    # Start with a generic greeting
    message = f"Dear {row['Custome Name']},\n\nThank you for purchasing the {item}.\n"
    
    # Personalize based on sentiment analysis
    if sentiment == 'POSITIVE':
        message += f"We're thrilled you're enjoying the {item}! "
    elif sentiment == 'NEGATIVE':
        message += f"We're sorry to hear about your experience with {item}. We're here to help. "
    else:  # NEUTRAL sentiment
        message += f"We hope you find the {item} to your liking. "
    
    # Additional recommendations or actions based on embeddings can be added here.
    # For this example, we'll keep it simple and skip this part.
    
    message += "Here are some products you might also like based on your purchase:\n\n[Recommended products]\n\nBest regards,\n[Your Company]"
    
    return message

df['Advanced Personalized Email'] = df.apply(generate_advanced_email, axis=1)

print(df[['Custome Name', 'Advanced Personalized Email']])


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\91981\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91981\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


        Custome Name                        Advanced Personalized Email
0       Ava Thompson  Dear Ava Thompson,\n\nThank you for purchasing...
1     Elliot Grayson  Dear Elliot Grayson,\n\nThank you for purchasi...
2       Lily Roberts  Dear Lily Roberts,\n\nThank you for purchasing...
3     Samuel Whitman  Dear Samuel Whitman,\n\nThank you for purchasi...
4    Grace Zimmerman  Dear Grace Zimmerman,\n\nThank you for purchas...
5       Henry Tallis  Dear Henry Tallis,\n\nThank you for purchasing...
6       Emily Fields  Dear Emily Fields,\n\nThank you for purchasing...
7   George Ellington  Dear George Ellington,\n\nThank you for purcha...
8     Rebecca Hollis  Dear Rebecca Hollis,\n\nThank you for purchasi...
9       Brian Oliver  Dear Brian Oliver,\n\nThank you for purchasing...
10     Maria Stilton  Dear Maria Stilton,\n\nThank you for purchasin...
11     Evan Richmond  Dear Evan Richmond,\n\nThank you for purchasin...
12   Sarah Patterson  Dear Sarah Patterson,\n\nThank you for pur

In [5]:
df.to_csv('n8nwithmails.csv')